In [2]:
import sys
import numpy as np
import impl.RNN as rnn
import impl.solver as solver

In [3]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [4]:
vocab_size = len(char_to_idx)

# hyper parameters
time_step = 30
n_iter = 13000 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [5]:
# import impl.RNN as nn
# net = nn.RNN(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

import impl.LSTM as nn
net = nn.LSTM(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# import impl.GRU as nn
# net = nn.GRU(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# import impl.GRU2 as nn
# net = nn.GRU2(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# import impl.GRU3 as nn
# net = nn.GRU3(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# # GRU4 goes to inf because the res-conn is gone.
# import impl.GRU4 as nn
# net = nn.GRU4(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [6]:
solver.adam_rnn(
    net, X, y,
    alpha=alpha,
    mb_size=time_step, # This is the width of the network
    n_iter=n_iter,
    print_after=print_after
)

Iter-1000 loss: 3.3522
he teesist and insti ate Sfevesd Sthesu a2nd momf'n ot tad tes"Gin momy ti6 old moeryetou" chee ina 


Iter-2000 loss: 2.6532
ten ainy hohe welaml Rf 99Gxe ingt ore nacs nlrentee n. P2pe Eo0t. whe wanutinas ticore se Jhporm un


Iter-3000 loss: 2.2468
wort HseoRe prom, ind whecorutine beon ug mare Japas Wo 2ky upure:Tid litn as eas, fisl erndendeires


Iter-4000 loss: 2.0849
s, ist ans. Japankeat in tof rokadd ar evin "s waredorad fot,rtho lyrar th6 wa to ina, Japengev of .


Iter-5000 loss: 1.9699
 lacon8 in lins fyof Wh4 whe fo wh roy55 phty an Wan 1851 xapandecity cminthe 193 the Sto, Japanih '


Iter-6000 loss: 1.8805
0151h9phl8 onsimin, ipunh byes ind lelou al hish cound bathe farth the Eas the Ea, estered bonsiom w


Iter-7000 loss: 1.7619
dedins tau and cop terold's somhmorer.58 6istomy iso mapred in caliWac ind in the Gtirnk "5 pevomulo


Iter-8000 loss: 1.6757
 ored eint. Japan of anctye countiy bolteld amnto. Thenouna-Japan esve perion as ecenecteden th